In [9]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt

#### Definições importantes que vão pro relatório
1. **Risk:** Risk takes on many forms but is broadly categorized as the chance an outcome or investment's actual return will differ from the expected outcome or return. Risk includes the possibility of losing some or all of the original investment.
2. **Beta:** By definition, the market has a beta of 1.0, and individual stocks are ranked according to how much they deviate from the market. A stock that swings more than the market over time has a beta above 1.0. If a stock moves less than the market, the stock's beta is less than 1.0. High-beta stocks are supposed to be riskier but provide a potential for higher returns; low-beta stocks pose less risk but also lower returns.
3. **Volatilidade:** Standard deviation measures the dispersion of data from its expected value. The standard deviation is used in making an investment decision to measure the amount of historical volatility associated with an investment relative to its annual rate of return. It indicates how much the current return is deviating from its expected historical normal returns. For example, a stock that has a high standard deviation experiences higher volatility, and therefore, a higher level of risk is associated with the stock.

### Fuzzy 1 - Compra/Venda de uma *stock*
1. **INPUTS**
    1. Porcentagem de crescimento
        * Universo: Quanto variou o preço da *stock* desde o último pico?
        * Conjunto Fuzzy: baixo, médio, alto
    2. Risco
        * Universo: Qual o risco de se investir nessa *stock* ?
        * Conjunto Fuzzy: baixo, médio, alto
2. **OUTPUTS**
    1. Porcentagem de compra/venda (é isso mesmo??)
        * Universo: Qual a porcentagem do *budget* que deve ser gasta com essa ação?
        * Conjunto Fuzzy: baixa, alta
3. **RULES**
    * Se a *stock* tem *alto crescimento* e *baixo risco* **ENTÃO** o investimento deve ser *alto*
    * Se a *stock* tem *alto crescimento* e *alto risco* **ENTÃO** o investimento deve ser *médio*
    * Se a *stock* tem *baixo crescimento* e *baixo risco* **ENTÃO** o investimento deve ser *médio*
    * Se a *stock* tem *baixo crescimento* e *alto risco* **ENTÃO** o investimento deve ser *baixo*
    * **OBS: DA PRA PENSAR EM MAIS REGRA**

### Fuzzy 2 - Classificação do Grau de Risco de uma *stock*
1. **INPUTS**
    1. Média de Variação (famosa volatilidade)
        * Universo: Qual foi a média de variação no valor das *stocks* (em módulo)?
        * Conjunto Fuzzy: baixo, médio, alto
    2. Beta
        * Universo: Qual o valor da volatilidade da *stock* com relação a volatilidade do mercado?
        * Conjunto Fuzzy: baixo, alto
2. **OUTPUTS**
    1. Risco
        * Universo: Qual o grau de risco ao se investir nessa *stock* ?
        * Conjunto Fuzzy: baixo, médio, alto
3. **RULES**
    * Se a *stock* apresenta *baixa variação* e um *baixo valor beta* **ENTAO** o risco é baixo
    * Se a *stock* apresenta *baixa variação* e um *alto valor beta* **ENTAO** o risco é baixo (ou médio)
    * Se a *stock* apresenta *alta variação* e um *alto valor *beta* **ENTAO** o risco é alto
    * Se a *stock* apresenta *alta variação* e um *baixo valor *beta* **ENTAO** o risco é alto (ou médio)
    * **OBS: DA PRA PENSAR EM MAIS REGRA**